# One big biome table

Since I am hoping to make comparisons of wood and leaf endophyte environmental patterns, I need to combine these datasets early in the biomformatics pipeline, to make them as comparable as possible. We'll try to stick to the [usearch (uparse)](http://drive5.com/usearch/) pipeline for the process, as much as possible.

<a id='environment'></a>

### Work environment

Working directory, on my machine:

In [2]:
cd /home/daniel/Documents/Taiwan_data/combined/combo_biome

We'll be using the [usearch (uparse)](http://drive5.com/usearch/) pipeline, version usearch v8.1.1861_i86linux32 on my personal machine, and the equivalent 64-bit version on our computing cluster [ACISS](http://aciss-computing.uoregon.edu/). These programs are abbreviated (soft linked) to "usearch81" and "usearch", respectively. 

<a id='mergepairs'></a> 

### Merging paired-end reads

Let's re-pair all readsets in the same manner, except for Roo's stromatal readset, which he aligned by hand. 

First, the leaf study reads include a split 6+6 bp barcode scheme for identifying reads, so these need to be clipped from one read and combined on the other. I wrote a python script for this, let's see if it works:

In [1]:
./BCunsplit4.py Roo_R2.fastq Roo_R1.fastq

This outputs two files, "rearranged_Roo_R2.fastq" and "rearranged_Roo_R2.fastq"

Next we trim a little to make sure we doing our alignments with high quality base calls. The sites for trimming are decided by looking at the raw reads [(see below)](#quality), and finding where quality begins to drop off. 
To trim, we'll use the [FASTX-toolkit](http://hannonlab.cshl.edu/fastx_toolkit/).

In [2]:
## wood
fastx_trimmer -l 255 -i woodR1.fastq -o woodR1_trimmed.fastq
fastx_trimmer -l 210 -i woodR2.fastq -o woodR2_trimmed.fastq

## leaves. These lengths were decided by Roo
fastx_trimmer -l 263 -i rearranged_Roo_R1.fastq -o Roo_R1_trimmed.fastq
fastx_trimmer -l 170 -i rearranged_Roo_R2.fastq -o Roo_R2_trimmed.fastq

Do the actual pairing:

In [4]:
## wood, let's pair both trimmed and untrimmed just to compare:
usearch -fastq_mergepairs woodR1.fastq -reverse woodR2.fastq  -fastqout woodtrimmedmerged.fastq -notrunclabels
usearch -fastq_mergepairs woodR1.fastq -reverse woodR2.fastq -fastqout woodmerged.fastq -notrunclabels

The "-notrunclabels" tag above asks usearch to keep the entire label of the forward reads, which is necessary because the wood reads, which are more recently sequenced than the leaf reads, contain sample info in their identifier lines. The leaves do not require this, their sample info is still in the sequence itself, to be use to [demultiplex](#demult) them, later.

** typical report from these:**

usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

<br>03:02 925Mb  100.0% 95.6% merged<br>5567799  Pairs (5.6M)<br>5323274  Merged (5.3M, 95.61%)<br>2067236  Alignments with zero diffs (37.13%)<br>0  Fwd tails Q <= 2 trimmed (0.00%)<br>15  Rev tails Q <= 2 trimmed (0.00%)<br>244525  No alignment found (4.39%)<br>0  Alignment too short (< 16) (0.00%)<br>4512849  Staggered pairs (81.05%) merged & trimmed<br>239.63  Mean alignment length
259.16  Mean merged read length<br>3.81  Mean fwd expected errors<br>6.27  Mean rev expected errors<br>1.57  Mean merged expected errors

These numbers look pretty good. Many of the erroneous reads will be taken out below, in a [quality filtering](#qf) step

In [5]:
## leaves
usearch -fastq_mergepairs Roo_R2.fastq -reverse Roo_R1.fastq -fastqout leafmerged.fastq
usearch -fastq_mergepairs Roo_R2_trimmed.fastq -reverse Roo_R1_trimmed.fastq -fastqout leaftrimmedmerged.fastq

What did trimming do to our reads? Let's take a look. To plot these we'll use the [fastx wrapper](http://hannonlab.cshl.edu/fastx_toolkit/commandline.html#fastq_quality_boxplot_usage) for [gnuplot](http://www.gnuplot.info/), which I've tinkered with just a little to change up the crowded axes of the original setup. Fastx requires that we first compile the quality data from the fastq files:

In [18]:
## wood quality stats:
fastx_quality_stats -i woodR1.fastq -o woodR1_fastxstats.txt
fastx_quality_stats -i woodR2.fastq -o woodR2_fastxstats.txt
fastx_quality_stats -i woodR1_trimmed.fastq -o woodR1_fastxstats.txt
fastx_quality_stats -i woodR2_trimmed.fastq -o woodR2_fastxstats.txt
fastx_quality_stats -i woodmerged.fastq -o woodmerged_fastxstats.txt
fastx_quality_stats -i woodtrimmedmerged.faestq -o woodtrimmedmerged_fastxstats.txt

In [ ]:
## leaf quality stats:
fastx_quality_stats -i Roo_R1.fastq -o Roo_R1_fastxstats.txt
fastx_quality_stats -i Roo_R2.fastq -o Roo_R2_fastxstats.txt
fastx_quality_stats -i Roo_R1_trimmed.fastq -o Roo_R1_trimmed_fastxstats.txt
fastx_quality_stats -i Roo_R2_trimmed.fastq -o Roo_R2_trimmed_fastxstats.txt
fastx_quality_stats -i leafmerged.fastq -o leafmerged_fastxstats.txt


Make the graphics:

In [20]:
## wood graphics
./dan_plot.sh -i woodR1_fastxstats.txt -o woodR1_quality.png
./dan_plot.sh -i woodR2_fastxstats.txt -o woodR2_quality.png
./dan_plot.sh -i  woodR1_trimmed_fastxstats.txt -o woodR1_trimmed_quality.png
./dan_plot.sh -i  woodR2_trimmed_fastxstats.txt -o woodR2_trimmed_quality.png
./dan_plot.sh -i woodmerged_fastxstats.txt -o woodmerged_quality.png
./dan_plot.sh -i  woodtrimmedmerged_fastxstats.txt -o woodtrimmedmerged_quality.png

## leaf graphics
./dan_plot.sh -i  Roo_R1_trimmed_fastxstats.txt -o Roo_R1_trimmed_quality.png
./dan_plot.sh -i  Roo_R2_trimmed_fastxstats.txt -o Roo_R2_trimmed_quality.png
./dan_plot.sh -i leaftrimmedmerged_fastxstats.txt -o leaftrimmedmerged_quality.png

<a id='quality'></a>

The **untrimmed wood R1** reads look like this:

<img src='woodR1_quality.png'>

Compare to **trimmed wood R1** reads:

<img src='woodR1_trimmed_quality.png'>

The **untrimmed wood R2** reads look like this:

<img src='woodR2_quality.png'>

Compare to **trimmed wood R2** reads:

<img src='woodR2_trimmed_quality.png'>

And the **untrimmed, merged wood** file looks like this:

<img src='woodmerged_quality.png'>

Obviously some problems here. So compare to the **merged, trimmed wood** reads:

<img src='woodtrimmedmerged_quality.png'>

Looks much better, but still a large dip in quality around 15 bp. More on this later.

Roo has already decided the trimming sites for his leaf data [(see above)](#mergepairs). Skip to the trimmed leaf R1 reads:

<img src='Roo_R1_trimmed_quality.png'>

The trimmed leaf R2 reads:

<img src='Roo_R2_trimmed_quality.png'>

Trimmed, merged leaf reads:

<img src='leaftrimmedmerged_quality.png'>

In both leaves and wood, I think we've vastly improved the situation by merging. The nice thing about using the usearch merging algorithms is that they use a bayesian approach to calculating the q-scores of paired reads, so that agreements on base calls between R1 and R2 improve Q scores, and disagreements reduce them. The reduction by disagreement is proportional to the confidence of the two base calls at a site, so if the disagreement occurs at the end of a read, where quality is lower, the higher (more reliable) Q has a much greater influence on the final q-score of a base call. Check out the explanation [here](http://drive5.com/usearch/manual/exp_errs.html).

<a id='qf'></a>

## Quality filtering

Continuing with the usearch pipeline, let's do some quality filtering. We'll use the [expected error approach](http://drive5.com/usearch/manual/exp_errs.html). We can set error cutoff of 1% of all bases in a read, meaning that a read of length 400 bp is thrown out if it likely contains 4 or more erroneous bases. I think this is permissable, given our OTU clustering will ultimately be done at 95% similarity.

In [25]:
## wood sequences
usearch -fastq_filter woodtrimmedmerged.fastq -fastq_maxee_rate .01 -fastqout wood_filtered.fastq

usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

01:04 857Mb  100.0% Filtering, 85.4% passed
   5323274  FASTQ recs (5.3M)              
   4548698  Converted (4.5M, 85.4%)

In [26]:
## leaves
usearch -fastq_filter leaftrimmedmerged.fastq -fastq_maxee_rate 0.01 -fastqout leaf_filtered.fastq

usearch v8.1.1803_i86linux64, 529Gb RAM, 32 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

07:01 2.4Gb  100.0% Filtering, 90.7% passed
  16701565  FASTQ recs (16.7M)             
  15145323  Converted (15.1M, 90.7%)

We can inspect these graphically as above, to see if avg read quality has improved. Not placing this here because the graphs look basically the same before and after filtering. But notice that we drop 15% of wood reads, and 9% of the leaf reads. 

<a id='fastq2fasta'></a>

## Convert from fastq to fasta format

Now that we've merged paired ends and done some quality filtering to ensure that we've hopefully mostly eliminated sequencer error (hah! see [index bleed](#bleed) below), let's convert to fasta as required by most downstream steps. Using FASTX toolkit again:

In [5]:
fastq_to_fasta -n -i leaf_filtered.fastq -o  leaf.fasta
fastq_to_fasta -n -i wood_filtered.fastq -o  wood.fasta

The "-n" flag tells fastx to retain sequences with "N" basecalls. Otherwise, these are removed, by default. Since ~1/2 of our leaf reads contain an "N", we need these. A single N basecall is an acceptable loss of information, OTU clustering and taxonomic assignments should be to deal with this. 

<a id="demult"></a>

## Demultiplex leaf reads

Roo's leaf reads were prepped at an earlier date than the wood reads. At the time of their sequencing, the standard method for denoting sample identities was to look for the presence of 12 bp golay barcodes, the ones that we cut and pasted when we [merged paired end reads](#mergepairs). For probably the only time in this pipeline, we will use a [qiime](http://qiime.org/) script, ["demultiplex_fasta.py"](http://qiime.org/scripts/demultiplex_fasta.html) that was made to parse samples by these golay barcodes.

This script requires a mapping file that lists the barcodes and their accompanying sample info, plus a "linkerprimersequence". I use a map file supplied by Roo. I do not know what a "linkerprimersequence" is, I believe this is supplied by the illumina software. The script seems to prefer .tsv format to .csv, and looks like this:

In [9]:
head leaf_sample_map.tsv

#SampleID	BarcodeSequence	LinkerPrimerSequence
1Leaf	ACCCATATATCC	GCTGCGTTCTTCATCGATGC
2Leaf	ACCCATAAGACG	GCTGCGTTCTTCATCGATGC
3Leaf	TCGCCAGAACCA	GCTGCGTTCTTCATCGATGC
4Leaf	ACCCATATCAAA	GCTGCGTTCTTCATCGATGC
5Leaf	ACCCATATAGTA	GCTGCGTTCTTCATCGATGC
6Leaf	ACCCATCTACAG	GCTGCGTTCTTCATCGATGC
7Leaf	ACCCATCATACC	GCTGCGTTCTTCATCGATGC
8Leaf	ACCCATCATTAT	GCTGCGTTCTTCATCGATGC
9Leaf	ACCCATCTATCT	GCTGCGTTCTTCATCGATGC


In [8]:
demultiplex_fasta.py -m leaf_sample_map.tsv -f leaf.fasta -o ./leaf_demult

This produces a folder with a log file and a file called "demultiplexed_seqs.fna". I will rename this to "leaf.fna" and bring it into our working directory. 

## Floating primers

In both of our read sets, "floating" primer sequences appear. This happens in other studies, as indicated by [Bálint et al. (2014)](http://onlinelibrary.wiley.com/doi/10.1002/ece3.1107/abstract;jsessionid=FBCBBBE428CAA870889926051DBC9927.f04t02). As advised by these authors, I remove the sequences that contain these floating primers with a script.